# 1. Imports of necessary libraries

In [50]:
import keras #High level interface of TensorFlow
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical 
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
from keras.optimizers import RMSprop
import tensorflowjs as tfjs
import pandas as pd
from sklearn.model_selection import train_test_split

# 2.Data importing& preprocessing

## 2.1 Importing and entry processing

In [51]:
train_set = pd.read_csv("../data/train.csv") # raw data

# DV
Y_train = train_set["label"]
# IV
X_train = train_set.drop(labels = ["label"], axis = 1)
#X_train

## 2.2Normalization to [0-1]  range

In [52]:
X_train = X_train / 255.0


X_train = X_train.values.reshape(-1,28,28,1)

Y_train = to_categorical(Y_train, num_classes = 10)

## 2.3Data spliting into train & valid set 

In [53]:
X_train, X_valid , Y_train , Y_valid = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 42)

## 2.4 Augmentation

In [54]:
igd = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range = 0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

igd.fit(X_train)

# 3. Building the architecture of Keras model & Compilation

## 3.1 Architecture

In [55]:
model = Sequential(
        [
            Conv2D(32, (3,3),activation = "relu",padding = 'Same',input_shape = (28,28,1)),
            Conv2D(32, (3,3),activation = "relu",padding = 'Same'),
            MaxPool2D(pool_size = (2, 2)),
            Dropout(0.2),
            Conv2D(32, (3,3), activation = "relu", padding ='Same'),
            Conv2D(32, (3,3),activation = "relu", padding = 'Same'),
            MaxPool2D(pool_size = (2, 2), strides = (2,2)),
            Dropout(0.25),
            Flatten(),
            Dense(units = 64, activation = "relu"),
            Dense(units = 10, activation = "sigmoid")  
        ]
)

"""# Create model using Keras Sequential API
model = Sequential()


# CONVOLUTIONAL/MAXPOOL LAYERS
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

# Convolutional 2D layer #1
# Computes 32 features using a 5x5 filter with ReLU activation.
model.add(Conv2D(
    filters = 32, 
    kernel_size = (5,5),
    padding = 'Same', 
    activation ='relu',
    input_shape = (28,28,1)
))

# Convolutional 2D layer #2
# Computes 32 features using a 5x5 filter with ReLU activation.
model.add(Conv2D(
    filters = 32, 
    kernel_size = (5,5),
    padding = 'Same', 
    activation ='relu'
))

# Pooling layer #1
# Max pooling layer with a 2x2 filter
model.add(MaxPool2D(pool_size=(2,2)))

# Dopout operation; 0.75 probability that element will be kept
model.add(Dropout(0.25))

# Convolutional 2D layer #3
# Computes 64 features using a 3x3 filter with ReLU activation.
model.add(Conv2D(
    filters = 64, 
    kernel_size = (3,3),
    padding = 'Same', 
    activation ='relu'
))

# Convolutional 2D layer #4
# Computes 64 features using a 3x3 filter with ReLU activation.
model.add(Conv2D(
    filters = 64, 
    kernel_size = (3,3),
    padding = 'Same', 
    activation ='relu'
))

# Pooling layer #1
# Max pooling layer with a 2x2 filter and stride of 2
model.add(MaxPool2D(
    pool_size=(2,2), 
    strides=(2,2)
))

# Dopout operation; 0.75 probability that element will be kept
model.add(Dropout(0.25))

# Flatten layer 
# Convert Tensor into a one single 1D vector
model.add(Flatten())

 
# FULLY CONNECTED LAYERS
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

# Danse layer #1
# Densely connected layer with 256 units
model.add(Dense(
    256, 
    activation = "relu"
))

# Dopout operation; 0.5 probability that element will be kept
model.add(Dropout(0.5))

# Danse layer #2
# Densely connected layer with 10 units
model.add(Dense(
    10, 
    activation = "softmax"
))"""

'# Create model using Keras Sequential API\nmodel = Sequential()\n\n\n# CONVOLUTIONAL/MAXPOOL LAYERS\n# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = \n\n# Convolutional 2D layer #1\n# Computes 32 features using a 5x5 filter with ReLU activation.\nmodel.add(Conv2D(\n    filters = 32, \n    kernel_size = (5,5),\n    padding = \'Same\', \n    activation =\'relu\',\n    input_shape = (28,28,1)\n))\n\n# Convolutional 2D layer #2\n# Computes 32 features using a 5x5 filter with ReLU activation.\nmodel.add(Conv2D(\n    filters = 32, \n    kernel_size = (5,5),\n    padding = \'Same\', \n    activation =\'relu\'\n))\n\n# Pooling layer #1\n# Max pooling layer with a 2x2 filter\nmodel.add(MaxPool2D(pool_size=(2,2)))\n\n# Dopout operation; 0.75 probability that element will be kept\nmodel.add(Dropout(0.25))\n\n# Convolutional 2D layer #3\n# Computes 64 features using a 3x3 filter with ReLU activation.\nmodel.add(Conv2D(\n    filters = 64, \n    kernel_size = (3,3),\n    pad

## 3.2 Compilation

In [56]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

# 4. Training

In [57]:
model.fit_generator(
    igd.flow(X_train, Y_train, batch_size = 128),
    epochs = 10,
    steps_per_epoch = X_train.shape[0] // 128,
    validation_data = (X_valid, Y_valid)
    )

Epoch 1/10
262/262 [==============================] - 126s 482ms/step - loss: 0.6353 - acc: 0.7934 - val_loss: 0.1007 - val_acc: 0.9714
Epoch 2/10
262/262 [==============================] - 134s 513ms/step - loss: 0.1816 - acc: 0.9449 - val_loss: 0.0742 - val_acc: 0.9782
Epoch 3/10
262/262 [==============================] - 136s 520ms/step - loss: 0.1308 - acc: 0.9605 - val_loss: 0.0538 - val_acc: 0.9837
Epoch 4/10
262/262 [==============================] - 132s 504ms/step - loss: 0.1039 - acc: 0.9675 - val_loss: 0.0514 - val_acc: 0.9849
Epoch 5/10
262/262 [==============================] - 129s 492ms/step - loss: 0.0879 - acc: 0.9730 - val_loss: 0.0400 - val_acc: 0.9882
Epoch 6/10
262/262 [==============================] - 127s 485ms/step - loss: 0.0791 - acc: 0.9759 - val_loss: 0.0345 - val_acc: 0.9880
Epoch 7/10
262/262 [==============================] - 128s 490ms/step - loss: 0.0704 - acc: 0.9784 - val_loss: 0.0331 - val_acc: 0.9896
Epoch 8/10
262/262 [============================

# Saving a model

In [49]:
tfjs.converters.save_keras_model(model, '../model')